In [1]:
from pathlib import Path
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torchvision
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch import nn
import time
from torch.utils.data.dataset import random_split
from tkinter import *





In [2]:
c=Counter()
df=pd.read_csv('/content/spam.csv')


In [3]:
tokenizer = get_tokenizer('basic_english')
for x in df["EmailText"]:
  tokens=tokenizer(x)
  tokens
  c.update(tokens)

In [4]:
res = [token for token in c.keys() if c[token] > 10]

arr = np.array
hash={}
count=0;
for i in res:
  hash[i]=count
  count=count+1
hash
hash["<unk>"]=count
hash.get(1)
res.append("<unk>")

In [5]:
clean_email_text=[]
for email in df["EmailText"]:
  clean_tokens=[]
  tokens=tokenizer(email)
  for token in tokens:
    if(token in res):
      clean_tokens.append(token)
    else:
      clean_tokens.append('<unk>')
  seperator=" "
  clean_email=seperator.join(clean_tokens)
  clean_email_text.append(clean_email)
df["CleanEmailText"]=clean_email_text
df.to_csv("/content/Clean_Spam.csv")

In [6]:
class Spam(Dataset):
    

    def __init__(self, datapath):
     # self.classes = pd.read_csv(annotations_file)#Labels?
      self.datapath=datapath

      
    def __getitem__(self, index):
      df=pd.read_csv(self.datapath)
      y,x=df.iloc[index]
      tokens=tokenizer(x)
      vectorized_x=[]
      for token in tokens:
        if(token in res):
          vectorized_x.append(hash.get(token))
        else:
          vectorized_x.append(hash.get("<unk>"))
      tensor_x=torch.tensor(vectorized_x,dtype=torch.int64)
      if(y=="ham"):
        label=torch.tensor(0,dtype=torch.int64)
      else:
        label=torch.tensor(1,dtype=torch.int64)
      return (tensor_x,label)

    def __len__(self):
      return (df[df.columns[0]].count())
  

In [7]:
data=Spam("/content/spam.csv")



In [8]:
val_set_size=((df[df.columns[0]].count()))*1.25
actual=((df[df.columns[0]].count()))*1
val_set_size=val_set_size-actual
train_set_size=actual-val_set_size
train_set_size+val_set_size


5572.0

In [9]:
train_dataset,val_dataset=torch.utils.data.random_split(data,[int(train_set_size),int(val_set_size)])

In [10]:
print("Length of train", len(train_dataset))
print("Length of Val",len(val_dataset))

Length of train 4179
Length of Val 1393


In [11]:
train_dataloader=DataLoader(train_dataset,batch_size=1,shuffle=True)
val_dataloader=DataLoader(val_dataset,batch_size=1)

In [12]:
class TextClassificationModel(nn.Module):
  def __init__(self,num_words,emb_size,num_class):
    super().__init__()
    self.emb_size=emb_size
    self.num_words=num_words
    self.emb1=nn.Embedding(self.num_words,self.emb_size)
    self.lstm1 =nn.LSTM(emb_size,hidden_size=32,batch_first=True)
    self.relu1=nn.ReLU()
    self.lin1 =nn.Linear(32,2)
  def forward(self,batched_data):
    token_embs=self.emb1(batched_data)
    outputs, (h_n,c_n)=self.lstm1(token_embs)
    last_hidden_state=h_n
    last_hidden_state = last_hidden_state.permute(1,0,2)
    last_hidden_state = last_hidden_state.flatten(start_dim=1)

    last_hidden_state = self.relu1(last_hidden_state)

    logits = self.lin1(last_hidden_state)

    return logits




In [13]:
num_class = 2
vocab_size = len(res)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class)
model

TextClassificationModel(
  (emb1): Embedding(991, 64)
  (lstm1): LSTM(64, 32, batch_first=True)
  (relu1): ReLU()
  (lin1): Linear(in_features=32, out_features=2, bias=True)
)

In [17]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()
    total_loss=0
    total_count=0
    for idx, (text, label) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text)
        loss = criterion(predited_label, label)
        total_loss+=loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        optimizer.zero_grad()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count+=label.size(0)

        if idx % log_interval == 0 and idx > 0:
            print('| epoch {0} | {1}/{2} batches | accuracy {3}| loss {4}'.format(epoch, idx, len(dataloader),
          total_acc/total_count,total_loss/log_interval))
            total_loss=0
            total_acc=0
            total_count=0

def evaluate(dataloader):
    total_loss=0
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, label) in enumerate(dataloader):
            predited_label = model(text)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss+=loss
    return total_loss/len(dataloader),total_acc/total_count

In [18]:
LR=.001
epochs=10
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [19]:
for epoch in range(1, epochs + 1):
  epoch_start=time.time()
  train(train_dataloader)
  val_loss,val_acc = evaluate(val_dataloader)
  print("validation | epoch {0}| accuracy {1} | loss {2} ".format(epoch,val_acc,val_loss))
 

| epoch 1 | 500/4179 batches | accuracy 0.9001996007984032| loss 0.33164170384407043
| epoch 1 | 1000/4179 batches | accuracy 0.948| loss 0.22682572901248932
| epoch 1 | 1500/4179 batches | accuracy 0.964| loss 0.1917949616909027
| epoch 1 | 2000/4179 batches | accuracy 0.954| loss 0.20536957681179047
| epoch 1 | 2500/4179 batches | accuracy 0.964| loss 0.19715990126132965
| epoch 1 | 3000/4179 batches | accuracy 0.956| loss 0.24606133997440338
| epoch 1 | 3500/4179 batches | accuracy 0.964| loss 0.18182267248630524
| epoch 1 | 4000/4179 batches | accuracy 0.97| loss 0.18844124674797058
validation | epoch 1| accuracy 0.9734386216798278 | loss 0.11363166570663452 
| epoch 2 | 500/4179 batches | accuracy 0.9900199600798403| loss 0.052089862525463104
| epoch 2 | 1000/4179 batches | accuracy 0.978| loss 0.13392199575901031
| epoch 2 | 1500/4179 batches | accuracy 0.98| loss 0.12994134426116943
| epoch 2 | 2000/4179 batches | accuracy 0.986| loss 0.06320465356111526
| epoch 2 | 2500/4179 ba

KeyboardInterrupt: ignored

In [22]:
val_loss,val_acc = evaluate(val_dataloader)
val_acc

0.9856424982053122

In [25]:
params=model.state_dict()
torch.save(parms,"trained_model_weights.pt")

In [27]:
loaded_params=torch.load("trained_model_weights.pt")
model.load_state_dict(loaded_params)

<All keys matched successfully>

In [ ]:
spam_labels = {2: "Spam", 1: "Not Spam"}
def predict(text):
  tokens=tokenizer(text)
  vector=[]
  for token in tokens:
    if token in res:
      vector.append(hash.get(token))
  t=torch.tensor(vector)
  output=model(text,torch.tensor([0]))
  return output.argmax(1).item()+1





In [32]:
string="Dear Nathan, Congratulations on your invitation to join Honor Society! Accepting this distinction to the largest academic society in the USA connects you with like-minded high achievers from your region and across the world, both in person and through our society's web portal. Honor Society recognizes achievement, and builds a framework for future success. Your satisfaction as a member is our highest priority and guaranteed. Now is a great time to secure your honors regalia for graduation, even if graduation is celebrated from home due to COVID-19. It's also a great time to obtain a print or free digital PDF version of your membership certificate, and be proud of your membership with exclusive member apparel. Honor Society regalia, t-shirts and caps are available exclusively for members in the Honor Society Store!  Join now to claim your place alongside other high-achievers. As an honor society inductee, you are eligible to receive our society regalia for graduation, or as a keepsake. Please activate your membership now"
tokens=tokenizer(string)
vector=[]
for token in tokens:
  if token in res:
    vector.append(hash.get(token))
t=torch.tensor(vector)
output=model(t,torch.tensor([0]))
f=output.argmax(1).item()+1
print(spam_labels.get(f))


TypeError: ignored

In [ ]:
print(hash)

In [ ]:
hash.get("go")

In [24]:
params=model.state_dict()
params

OrderedDict([('emb1.weight',
              tensor([[ 0.3002, -0.6702, -1.3321,  ..., -1.7519,  0.4022, -0.6568],
                      [ 1.2215, -0.7221,  0.3612,  ...,  0.8680,  0.3894,  1.4103],
                      [-0.7370,  0.9719,  0.6571,  ..., -0.5504,  0.9351, -1.1052],
                      ...,
                      [ 1.0293, -1.0766, -1.1664,  ...,  1.2490,  0.8252, -0.9720],
                      [-0.1692, -1.1205,  1.0060,  ..., -1.4079,  0.5251,  0.9065],
                      [ 2.1366,  0.2230, -0.2871,  ..., -0.4251, -1.1195, -1.6365]])),
             ('lstm1.weight_ih_l0',
              tensor([[ 0.1875, -0.1929, -0.0908,  ..., -0.0941, -0.2039, -0.2603],
                      [-0.1869,  0.1580,  0.1210,  ...,  0.0601,  0.2309, -0.0668],
                      [ 0.1860, -0.0249,  0.0979,  ..., -0.2043, -0.0242, -0.0690],
                      ...,
                      [ 0.1256,  0.1671, -0.0026,  ...,  0.0441, -0.0811, -0.2452],
                      [ 0.2882, -0.197